In [0]:
from pyspark.sql.functions import col
from pyspark.sql import DataFrame

# Configurações de conexão com o PostgreSQL
jdbc_url = "jdbc:postgresql://dpg-cuciu352ng1s73b52bgg-a.ohio-postgres.render.com:5432/transactions_yz0m"
jdbc_properties = {
    "user": "transactions_yz0m_user",
    "password": "66BsNmXaNh3btK1ZY8GLbnrFmwUFK9xY",
    "driver": "org.postgresql.Driver"
}

# Nome das tabelas
postgres_table = "transactions"
raw_table = "raw_transactions"
final_table = "transactions"

# Função para carregar dados incrementais do PostgreSQL
def load_incremental_data() -> DataFrame:
    # Verificar o último registro em raw_transactions
    if spark.catalog.tableExists(raw_table):
        last_transaction_date = spark.sql(f"SELECT MAX(transaction_date) FROM {raw_table}").collect()[0][0]
        if last_transaction_date:
            print(f"Última transação registrada: {last_transaction_date}")
            query = f"""
                (SELECT * FROM {postgres_table} WHERE transaction_date > '{last_transaction_date}') AS t
            """
        else:
            query = f"(SELECT * FROM {postgres_table}) AS t"
    else:
        print("Tabela 'raw_transactions' não encontrada. Criando pela primeira vez.")
        query = f"(SELECT * FROM {postgres_table}) AS t"

    # Carregar dados do PostgreSQL
    return (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", query)
        .option("user", jdbc_properties["user"])
        .option("password", jdbc_properties["password"])
        .option("driver", jdbc_properties["driver"])
        .load()
    )

# Atualizar a tabela raw_transactions
def update_raw_transactions():
    # Carregar os dados incrementais
    incremental_data = load_incremental_data()

    if incremental_data.count() > 0:
        print(f"Inserindo {incremental_data.count()} novos registros em 'raw_transactions'...")
        incremental_data.write.format("delta").mode("append").saveAsTable(raw_table)
        print("Atualização de 'raw_transactions' concluída.")
    else:
        print("Nenhum dado novo encontrado para atualizar em 'raw_transactions'.")

# Atualizar a tabela final transactions
def update_transactions():
    if spark.catalog.tableExists(raw_table):
        print("Atualizando a tabela final 'transactions' com os dados de 'raw_transactions'...")
        raw_data = spark.table(raw_table)
        raw_data.write.format("delta").mode("overwrite").saveAsTable(final_table)
        print("Tabela 'transactions' atualizada com sucesso.")
    else:
        print("A tabela 'raw_transactions' não existe. Não há dados para atualizar.")

# Executar o pipeline
if __name__ == "__main__":
    update_raw_transactions()
    update_transactions()
